# Midland Basin Well Data

## Import dependencies and load data

In [1]:
import pandas as pd
import numpy as np

In [51]:
# CSV Path
path = "Resources\Midland Basin Wells Header Data_Clean.csv"

In [52]:
# Load the data as a dataframe
df = pd.read_csv(path)
df.head()

,Well ID,Lease Name,Well Number,Operator,County,Landing Zone,Surf Lat,Surf Long,BH Lat,BH Long,...,Date Completed,Total Fluid (bbl),Total Proppant (lb),"Well Spacing, ft",Well Name,Fluid (bbl/ft),Prop (lb/ft),Avg PPG,"Oil EUR, Mbbl","Gas EUR, MMcf"
0,42-329418360000,BRADFORD TRUST A UNIT 3,1513AH,XTO ENERGY INC,MIDLAND,WCMP A,31.738410,-102.009040,31.715650,-102.003820,...,2018-10-05,17406252.0,10804000.0,NaN,BRADFORD TRUST A UNIT 3 1513AH,54.0,1412.0,0.62,208.0,3005.0
1,42-329418710000,WARFIELD EAST C,103LS,DIAMONDBACK EXPLORATION & PROD LLC,MIDLAND,SPBYL SH,31.922233,-102.224514,31.892186,-102.214750,...,2018-06-05,NaN,NaN,NaN,WARFIELD EAST C 103LS,NaN,NaN,NaN,758.0,932.0
2,42-329418760000,ST,4043WA,DIAMONDBACK EXPLORATION & PROD LLC,MIDLAND,WCMP A,31.982514,-102.211939,31.955094,-102.205336,...,2018-05-25,15855777.0,16090000.0,NaN,ST 4043WA,37.0,1585.0,1.01,456.0,1758.0
3,42-329418860000,MUSTANG,H120UH,PERMIAN DEEP ROCK OIL CO LLC,MIDLAND,WCMP D,31.972125,-102.067947,31.997833,-102.076536,...,2018-09-05,16382508.0,20288000.0,NaN,MUSTANG H120UH,42.0,2197.0,1.24,858.0,1556.0
4,42-329418890000,GBG 41/38 UNIT,0032LS,CHEVRON U S A INC,MIDLAND,SPBYL SH,31.853160,-102.129826,31.879517,-102.140645,...,2018-06-26,21463544.0,20104000.0,NaN,GBG 41/38 UNIT 0032LS,52.0,2052.0,0.94,759.0,1338.0


In [53]:
# Summarize initial dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14703 entries, 0 to 14702
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Well ID              14703 non-null  object 
 1   Lease Name           14703 non-null  object 
 2   Well Number          14703 non-null  object 
 3   Operator             14703 non-null  object 
 4   County               14703 non-null  object 
 5   Landing Zone         14176 non-null  object 
 6   Surf Lat             14703 non-null  float64
 7   Surf Long            14703 non-null  float64
 8   BH Lat               14309 non-null  float64
 9   BH Long              14309 non-null  float64
 10  Date Spud            14392 non-null  object 
 11  Lateral Len (ft)     12126 non-null  float64
 12  Date Completed       12747 non-null  object 
 13  Total Fluid (bbl)    11634 non-null  float64
 14  Total Proppant (lb)  11541 non-null  float64
 15  Well Spacing, ft     0 non-null     

## Data Cleaning

In [54]:
# Drop the columns where all values are null
sel_columns = df[["Well ID","Well Name","Landing Zone","Surf Lat",'Surf Long',"BH Lat","BH Long"]]
dist_df = sel_columns.copy()

## Main Code

In [55]:
dist_df["Mid Point Lat"] = (dist_df["Surf Lat"] + dist_df["BH Lat"])/2
dist_df["Mid Point Long"] = (dist_df["Surf Long"] + dist_df["BH Long"])/2
dist_df["Left Well"] = ""
dist_df["Right Well"] = ""
dist_df["Left Distance"] = 0
dist_df["Right Distance"] = 0
dist_df["Final Distance"] = 0
dist_df.head()

,Well ID,Well Name,Landing Zone,Surf Lat,Surf Long,BH Lat,BH Long,Mid Point Lat,Mid Point Long,Left Well,Right Well,Left Distance,Right Distance,Final Distance
0,42-329418360000,BRADFORD TRUST A UNIT 3 1513AH,WCMP A,31.738410,-102.009040,31.715650,-102.003820,31.727030,-102.006430,,,0,0,0
1,42-329418710000,WARFIELD EAST C 103LS,SPBYL SH,31.922233,-102.224514,31.892186,-102.214750,31.907210,-102.219632,,,0,0,0
2,42-329418760000,ST 4043WA,WCMP A,31.982514,-102.211939,31.955094,-102.205336,31.968804,-102.208638,,,0,0,0
3,42-329418860000,MUSTANG H120UH,WCMP D,31.972125,-102.067947,31.997833,-102.076536,31.984979,-102.072242,,,0,0,0
4,42-329418890000,GBG 41/38 UNIT 0032LS,SPBYL SH,31.853160,-102.129826,31.879517,-102.140645,31.866338,-102.135235,,,0,0,0


In [56]:
well_dist_df = pd.DataFrame(columns=['Distance','Well Name'])
well_dist_df['Distance'].astype(float)
for index,rows in dist_df.iterrows():
    # Current well lat, long and landing zone
    lat1 = dist_df.at[index,'Mid Point Lat']
    lon1 = dist_df.at[index,'Mid Point Long']
    lz = dist_df.at[index,'Landing Zone']
    
    # print(index,lz)
    
    # Filter dataframe only for same landing zone
    filt_df = dist_df[dist_df['Landing Zone'] == lz]
    filt_df.reset_index(inplace=True)
    
    #print(filt_df)
    
    # Dataframe will hold all distances calculated
    well_dist_df['Well Name'] = filt_df['Well Name'].copy()
    well_dist_df['Distance'] = calc_dist(lon1,lat1,filt_df['Mid Point Long'],filt_df['Mid Point Lat']) # Calculate Distance from func
    
    # print(well_dist_df)
    
    # Sort distances ascending
    well_dist_df.sort_values(by='Distance',inplace=True,ignore_index=True)
    
    # Select closest wells
    for i,dist in well_dist_df.iterrows():
        temp_dist = well_dist_df.at[i,'Distance']
        current_well = well_dist_df.at[0,'Well Name']
        # assuming closest wells are going to be >250 ft
        if temp_dist>200:
            left_dist = temp_dist
            if (pd.isnull(left_dist) or left_dist > 2000):
                left_dist = 2000
            left_well = well_dist_df.at[i,'Well Name']
            right_dist = well_dist_df.at[i+1,'Distance']
            if (pd.isnull(right_dist) or right_dist > 2000):
                right_dist = 2000
            right_well = well_dist_df.at[i+1,'Well Name']
            break
    final_dist = (left_dist + right_dist)/2
    dist_df.loc[index,"Left Well"] = left_well
    dist_df.loc[index,"Right Well"] = right_well
    dist_df.loc[index,"Left Distance"] = left_dist
    dist_df.loc[index,"Right Distance"] = right_dist
    dist_df.loc[index,"Final Distance"] = final_dist
dist_df.head(15)

,Well ID,Well Name,Landing Zone,Surf Lat,Surf Long,BH Lat,BH Long,Mid Point Lat,Mid Point Long,Left Well,Right Well,Left Distance,Right Distance,Final Distance
0,42-329418360000,BRADFORD TRUST A UNIT 3 1513AH,WCMP A,31.738410,-102.009040,31.715650,-102.003820,31.727030,-102.006430,BRADFORD TRUST A UNIT 3 1514AH,BRADFORD TRUST A UNIT 3 1512AH,385.578227,1020.986620,703.282424
1,42-329418710000,WARFIELD EAST C 103LS,SPBYL SH,31.922233,-102.224514,31.892186,-102.214750,31.907210,-102.219632,WARFIELD EAST B 102LS,WARFIELD EAST A 101LS,326.536360,646.354653,486.445507
2,42-329418760000,ST 4043WA,WCMP A,31.982514,-102.211939,31.955094,-102.205336,31.968804,-102.208638,ST 4044WA,ST 4045WA,327.992756,793.143335,560.568046
3,42-329418860000,MUSTANG H120UH,WCMP D,31.972125,-102.067947,31.997833,-102.076536,31.984979,-102.072242,CHARGER H120UH,REBEL H115UX,2000.000000,2000.000000,2000.000000
4,42-329418890000,GBG 41/38 UNIT 0032LS,SPBYL SH,31.853160,-102.129826,31.879517,-102.140645,31.866338,-102.135235,GBG 41/38 UNIT 0034LS,GBG 41/38 UNIT 0036LS,486.268801,999.908827,743.088814
5,42-329418900100,GBG 41/38 UNIT 0033WB,WCMP A,31.853179,-102.129748,31.880202,-102.137624,31.866690,-102.133686,GBG 41/38 UNIT 0035WB,SCHARBAUER C JR 4223H,496.543149,2000.000000,1248.271574
6,42-329419010000,GWENDOLYN 2622LA,SPBYL SD,31.875750,-102.244552,31.898461,-102.250973,31.887105,-102.247763,GWENDOLYN 2611LA,GWENDOLYN 2525LA,1205.497160,2000.000000,1602.748580
7,42-329419370000,HOGAN 1-13-D 2808H,SPBYL SH,32.053328,-102.073928,32.033210,-102.066620,32.043269,-102.070274,HOGAN 1-13-C 2806H,HOGAN 1-24-E 2810H,333.265866,1601.191061,967.228464
8,42-329419550000,BANAY BA 02WB,WCMP B,31.672449,-102.248621,31.700191,-102.256276,31.686320,-102.252448,BANAY BC 06WB,BANAY 10WB,908.330566,1566.829334,1237.579950
9,42-329419580000,RIO SAN MARCOS 36-45 UNIT 1 242,NaN,31.791895,-102.005341,NaN,NaN,NaN,NaN,BANAY BC 06WB,BANAY 10WB,908.330566,1566.829334,1237.579950


In [57]:
dist_df.to_csv('Resources\Dist Output_MP.csv',index=False)

In [36]:
def calc_dist(lon1, lat1,lon2,lat2):
    
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    newlon = lon2 - lon1
    newlat = lat2 - lat1

    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2

    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    ft = 3958 * 5280 * dist # 6367 for distance in KM for miles use 3958
    return ft